In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchsampler import ImbalancedDatasetSampler
import matplotlib.pyplot as plt
import os
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from functools import reduce
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import sklearn 
from torch.autograd import Variable

In [2]:
import loaders as l
import models as m
import train_eval as te

In [3]:
import cv2

In [4]:

from ImageClassification import helpers as h
import argparse
import time

In [5]:
saved_model_path='C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/firstModel'
net = m.BaseNet_v2()
net.load_state_dict(torch.load(saved_model_path))

<All keys matched successfully>

In [12]:
# load the image and define the window width and height
from PIL import Image
from matplotlib import cm
image = cv2.imread("C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/test3.jpg")
(winW, winH) = (36, 36)
transform = transforms.Compose(
    [transforms.Grayscale(), 
     transforms.ToTensor(), 
     transforms.Normalize(mean=(0,),std=(1,))])
face_detected=0
layer=0
w=0
color_face=(255,0,0)
color_noFace=(0, 255, 0)
sleep=0.005

# loop over the image pyramid
for resized in h.pyramid(image, scale=1.2):
    # loop over the sliding window for each layer of the pyramid
    layer+=1
    for (x, y, window) in h.sliding_window(resized, stepSize=20, windowSize=(winW, winH)):
        # if the window does not meet our desired window size, ignore it
        if window.shape[0] != winH or window.shape[1] != winW:
            continue

        # THIS IS WHERE YOU WOULD PROCESS YOUR WINDOW, SUCH AS APPLYING A
        # MACHINE LEARNING CLASSIFIER TO CLASSIFY THE CONTENTS OF THE
        # WINDOW
        w+=1
        im = Image.fromarray(window.astype('uint8'), 'RGB')
        im=transform(im)
        im = im.unsqueeze(0)
        outputs=net(im)
        
        p=torch.nn.functional.softmax(outputs, dim=1).sum().detach().numpy()
        #print(p)
        y_pred_softmax = torch.log_softmax(outputs, dim = 1)
        _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
        o=np.array(y_pred_tags)
 
        color=color_noFace
        sleep=0.005
        if o==1:
            face_detected+=1
            """
            print(layer)
            print(w)
            print(p)
            print(o)
            print()
            """
            color=color_face
            sleep=1
        # since we do not have a classifier, we'll just draw the window
        clone = resized.copy()
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), color, 2)
        cv2.imshow("Window", clone)
        cv2.waitKey(1)
        time.sleep(sleep)
    w=0
cv2.waitKey(1)
#cv2.destroyAllWindows() 
print(face_detected)

36
